<a href="https://colab.research.google.com/github/velasoka-repo/EVA4/blob/master/Session-7/EVA4_S7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Github Link](https://github.com/velasoka-repo/EVA4/new/master/Session-7)

# Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
EVA4_PATH = "/content/gdrive/My Drive/Colab Notebooks/EVA4/"
import sys
if not EVA4_PATH in sys.path:
  sys.path.append(EVA4_PATH)
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/EVA4/']

# Python Module Import

In [0]:
import velasoka
device = velasoka.get_available_device()

# Simple Model
- - -
**Uses** C1, C2, C3, C4 & O
* Conv2d
* BatchNorm2d
* ReLU
* MaxPool2d
* AvgPool2d

In [0]:
import torch.nn as nn
class SimpleModel(nn.Module):
    def __init__(self, fm_size):
        super(SimpleModel, self).__init__()
        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=fm_size, kernel_size=3, padding=1), #i=32, o=32
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=3, padding=1), #i=32, o=32
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #i=32, o=16
        )
        fm_size *=2 #64
        self.c2 = nn.Sequential(
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1), #i=16, o=16
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=3, padding=1), #i=16, o=16
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #i=16, o=8
        )
        fm_size *=2 #128
        self.c3 = nn.Sequential(
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3), #i=8, o=6
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=3),  #i=6, o=4
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  #i=4, o=2
        )
        fm_size *=2 #256
        self.c4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=2), #i=4, o=2
            nn.Conv2d(in_channels=fm_size, out_channels=10, kernel_size=1) #i=6, o=4
        )

    def forward(self, x):
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)
        x = x.view(-1, 10)
        return x

## Run

In [0]:
#model
simple_model = SimpleModel(fm_size=32).to(device)

#dataset, data_loader
dataset = velasoka.get_cifar10_dataset()
data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

#optimizer, loss_function
optimizer = velasoka.get_sgd_optimizer(model=simple_model)
loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

#model_summary
velasoka.model_summary(model=simple_model, input_size=(3, 32, 32), device=device)
#start cnn
velasoka.start_cnn(model=simple_model, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=8)

#save model
drive_file_path = velasoka.get_drive_path(folder="Session-7", filename="simple_model.pt")
velasoka.save_model(model=simple_model, path=drive_file_path)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          18,496
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,928
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]          73,856
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12

Training Batch=12499, loss=1.14653, Correct Prediction=25956/50000, Train Accuracy=51.91200: 100%|██████████| 12500/12500 [01:59<00:00, 104.50it/s]
Test Batch=2499, Correct Validation=7058/10000, Test Accuracy=70.58000: 100%|██████████| 2500/2500 [00:15<00:00, 164.84it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.20971, Correct Prediction=40769/50000, Train Accuracy=81.53800: 100%|██████████| 12500/12500 [02:02<00:00, 101.71it/s]
Test Batch=2499, Correct Validation=8114/10000, Test Accuracy=81.14000: 100%|██████████| 2500/2500 [00:14<00:00, 166.76it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 6


Training Batch=12499, loss=0.65215, Correct Prediction=41981/50000, Train Accuracy=83.96200: 100%|██████████| 12500/12500 [01:59<00:00, 104.19it/s]
Test Batch=2499, Correct Validation=8188/10000, Test Accuracy=81.88000: 100%|██████████| 2500/2500 [00:14<00:00, 173.57it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 7


Training Batch=12499, loss=0.75351, Correct Prediction=43143/50000, Train Accuracy=86.28600: 100%|██████████| 12500/12500 [01:57<00:00, 106.51it/s]
Test Batch=2499, Correct Validation=8186/10000, Test Accuracy=81.86000: 100%|██████████| 2500/2500 [00:14<00:00, 166.77it/s]


# Dilated Convolution
- - -
**Uses** C1, C2, C3, C4 & O
* Conv2d
* BatchNorm2d
* ReLU
* MaxPool2d
* AvgPool2d
* dilation=2

In [0]:
import torch.nn as nn
class DilatedModel(nn.Module):
    def __init__(self, fm_size):
        super(DilatedModel, self).__init__()
        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=fm_size, kernel_size=3, padding=1), #i=32, o=32
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=3, padding=1), #i=32, o=32
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #i=32, o=16
        )
        fm_size *=2 #64
        self.c2 = nn.Sequential(
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1), #i=16, o=16
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=3, padding=1), #i=16, o=16
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #i=16, o=8
        )
        fm_size *=2 #128
        self.c3 = nn.Sequential(
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1, dilation=2), #i=8, o=5
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=2),  #i=5, o=4
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  #i=4, o=2
        )
        fm_size *=2 #256
        self.c4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=2), #i=4, o=2
            nn.Conv2d(in_channels=fm_size, out_channels=10, kernel_size=1) #i=6, o=4
        )

    def forward(self, x):
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)
        x = x.view(-1, 10)
        return x

## Run

In [0]:
#model
dilated_model = DilatedModel(fm_size=32).to(device)

#dataset, data_loader
dataset = velasoka.get_cifar10_dataset()
data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

#optimizer, loss_function
optimizer = velasoka.get_sgd_optimizer(model=dilated_model)
loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

#model_summary
velasoka.model_summary(model=dilated_model, input_size=(3, 32, 32), device=device)
#start cnn
velasoka.start_cnn(model=dilated_model, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=8)

#save model
drive_file_path = velasoka.get_drive_path(folder="Session-7", filename="dilated_model.pt")
velasoka.save_model(model=dilated_model, path=drive_file_path)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          18,496
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,928
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]          73,856
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12

Training Batch=12499, loss=0.77478, Correct Prediction=29111/50000, Train Accuracy=58.22200: 100%|██████████| 12500/12500 [01:56<00:00, 107.65it/s]
Test Batch=2499, Correct Validation=7221/10000, Test Accuracy=72.21000: 100%|██████████| 2500/2500 [00:14<00:00, 173.86it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.00231, Correct Prediction=42963/50000, Train Accuracy=85.92600: 100%|██████████| 12500/12500 [01:58<00:00, 105.85it/s]
Test Batch=2499, Correct Validation=8224/10000, Test Accuracy=82.24000: 100%|██████████| 2500/2500 [00:14<00:00, 172.33it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 6


Training Batch=12499, loss=2.08339, Correct Prediction=43989/50000, Train Accuracy=87.97800: 100%|██████████| 12500/12500 [01:57<00:00, 106.74it/s]
Test Batch=2499, Correct Validation=8169/10000, Test Accuracy=81.69000: 100%|██████████| 2500/2500 [00:14<00:00, 169.44it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 7


Training Batch=12499, loss=0.77143, Correct Prediction=44951/50000, Train Accuracy=89.90200: 100%|██████████| 12500/12500 [01:56<00:00, 107.19it/s]
Test Batch=2499, Correct Validation=8353/10000, Test Accuracy=83.53000: 100%|██████████| 2500/2500 [00:14<00:00, 171.97it/s]


# Dilated, Depthwise Separable Convolution
- - -
**Uses** C1, C2, C3, C4 & O
* Conv2d
* BatchNorm2d
* ReLU
* MaxPool2d
* AvgPool2d
* dilation=2
* groups=`<in_channel>`

In [0]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, fm_size):
        super(Net, self).__init__()
        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=fm_size, kernel_size=3, padding=1), #i=32, o=32
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=3, padding=1), #i=32, o=32
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #i=32, o=16
        )
        fm_size *=2 #64
        self.c2 = nn.Sequential(
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1), #i=16, o=16
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1, groups=fm_size), #i=16, o=16
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=1, padding=1), #i=16, o=16
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #i=16, o=8
        )
        fm_size *=2 #128
        self.c3 = nn.Sequential(
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1, dilation=2), #i=8, o=5
            nn.BatchNorm2d(num_features=fm_size),
            nn.ReLU(),
            nn.Conv2d(in_channels=fm_size, out_channels=fm_size*2, kernel_size=2),  #i=5, o=4
            nn.BatchNorm2d(num_features=fm_size*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  #i=4, o=2
        )
        fm_size *=2 #256
        self.c4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=2), #i=4, o=2
            nn.Conv2d(in_channels=fm_size, out_channels=10, kernel_size=1) #i=6, o=4
        )

    def forward(self, x):
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)
        x = x.view(-1, 10)
        return x

# Run

In [0]:
#model
net = Net(fm_size=32).to(device)

#dataset, data_loader
dataset = velasoka.get_cifar10_dataset()
data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

#optimizer, loss_function
optimizer = velasoka.get_sgd_optimizer(model=net)
loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

#model_summary
velasoka.model_summary(model=net, input_size=(3, 32, 32), device=device)
#start cnn
velasoka.start_cnn(model=net, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=8)

#save model
drive_file_path = velasoka.get_drive_path(folder="Session-7", filename="net.pt")
velasoka.save_model(model=net, path=drive_file_path)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          18,496
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,928
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]             640
           Conv2d-12          [-1, 128, 18, 18]           8,320
      BatchNorm2d-13          [-1, 128, 18, 18]             256
             ReLU-14          [-1, 128,

Training Batch=12499, loss=0.37381, Correct Prediction=28970/50000, Train Accuracy=57.94000: 100%|██████████| 12500/12500 [02:01<00:00, 103.14it/s]
Test Batch=2499, Correct Validation=7076/10000, Test Accuracy=70.76000: 100%|██████████| 2500/2500 [00:14<00:00, 168.52it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.41091, Correct Prediction=42652/50000, Train Accuracy=85.30400: 100%|██████████| 12500/12500 [02:01<00:00, 103.28it/s]
Test Batch=2499, Correct Validation=8117/10000, Test Accuracy=81.17000: 100%|██████████| 2500/2500 [00:14<00:00, 171.34it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 6


Training Batch=12499, loss=1.80227, Correct Prediction=43696/50000, Train Accuracy=87.39200: 100%|██████████| 12500/12500 [02:00<00:00, 103.96it/s]
Test Batch=2499, Correct Validation=8184/10000, Test Accuracy=81.84000: 100%|██████████| 2500/2500 [00:14<00:00, 169.38it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 7


Training Batch=12499, loss=0.34167, Correct Prediction=44737/50000, Train Accuracy=89.47400: 100%|██████████| 12500/12500 [02:00<00:00, 103.55it/s]
Test Batch=2499, Correct Validation=8250/10000, Test Accuracy=82.50000: 100%|██████████| 2500/2500 [00:14<00:00, 174.72it/s]
